[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/session-4/loops-with-langchain-and-langgraph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239974-lesson-4-loops-with-langchain-and-langgraph)

# Loops with LangChain vs LangGraph

## 🔄 The Challenge: Iterative Information Gathering

This notebook demonstrates a fundamental limitation of **LangChain** and shows how **LangGraph** naturally handles **loops and iterations**.

### 🎯 Real-World Scenario: Iterative Research Assistant
**Build an AI that conducts thorough research by:**

1. 📝 **Taking a user's research question**
2. 🔍 **Searching the web for initial information**
3. 🤔 **Analyzing if the information is sufficient**
4. 🔄 **If not sufficient: generating follow-up search queries and repeating**
5. ✅ **When sufficient: providing comprehensive final answer**
6. 👤 **Asking user if they want more detail on any aspect**
7. 🔁 **Looping back to gather more specific information if requested**

### 💭 Why This Matters:
Real research is **iterative**:
- Initial search reveals knowledge gaps
- New questions emerge from partial answers
- Users want to drill down into specific aspects
- Quality research requires **multiple search-analyze cycles**

Let's see how each framework handles this...

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python

## Setup

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Verify the keys are loaded (optional - remove in production)
print("✅ Environment variables loaded:")
print(f"OPENAI_API_KEY: {'✓ Set' if os.environ.get('OPENAI_API_KEY') else '✗ Missing'}")
print(f"TAVILY_API_KEY: {'✓ Set' if os.environ.get('TAVILY_API_KEY') else '✗ Missing'}")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Dict, List, Annotated
import operator
import time

# Initialize LLM and search tool
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
search_tool = TavilySearchResults(max_results=3)

# 🔗 A. LangChain Approach: No Native Loop Support

## 😫 The Problem: Linear Chains Can't Loop Back

In [ ]:
# LangChain: Attempting to create iterative research (doesn't work well)
class LangChainIterativeResearch:
    def __init__(self):
        # Chain 1: Initial search
        self.search_prompt = PromptTemplate(
            input_variables=["question"],
            template="Generate a search query for: {question}"
        )
        
        # Chain 2: Analyze completeness
        self.analysis_prompt = PromptTemplate(
            input_variables=["question", "search_results"],
            template="""Question: {question}
Search Results: {search_results}

Is this information sufficient to fully answer the question? 
If not, what specific aspects need more research?

Respond with:
SUFFICIENT: Yes/No
GAPS: [list any information gaps]
NEXT_QUERY: [suggest next search query if not sufficient]"""
        )
        
        self.analysis_chain = LLMChain(llm=llm, prompt=self.analysis_prompt)
        
        # Chain 3: Final answer
        self.answer_prompt = PromptTemplate(
            input_variables=["question", "all_results"],
            template="""Question: {question}
All Research Results: {all_results}

Provide a comprehensive answer based on all the research:"""
        )
        
        self.answer_chain = LLMChain(llm=llm, prompt=self.answer_prompt)
    
    def research_with_manual_loops(self, question: str, max_iterations: int = 10):
        """Attempt iterative research with manual while loops - HACKY!"""
        print(f"🔍 Starting research on: {question}")
        
        all_results = []
        current_query = question
        iteration = 0
        
        # ❌ PROBLEM: Manual while loop - not part of LangChain's design!
        while iteration < max_iterations: #max_retry
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")
            print(f"🔍 Searching: {current_query}")
            
            # Search
            try:
                search_results = search_tool.invoke(current_query)
                formatted_results = "\n\n".join([
                    f"Source: {result['url']}\n{result['content']}"
                    for result in search_results
                ])
                all_results.append(formatted_results)
            except Exception as e:
                print(f"⚠️ Search failed: {e}")
                break
            
            # ❌ PROBLEM: Chain can't decide to loop - we force it with external logic
            analysis = self.analysis_chain.run(
                question=question, 
                search_results=formatted_results
            )
            
            print(f"📊 Analysis: {analysis[:200]}...")
            
            # ❌ PROBLEM: Crude parsing to determine if we should continue
            if "SUFFICIENT: Yes" in analysis or "Yes" in analysis.split('\n')[0]:
                print("✅ Research deemed sufficient")
                break
            
            # ❌ PROBLEM: Manually extract next query - very brittle!
            lines = analysis.split('\n')
            next_query = current_query  # Default fallback
            for line in lines:
                if "NEXT_QUERY:" in line:
                    next_query = line.split("NEXT_QUERY:")[1].strip()
                    break
            
            current_query = next_query
            
            # ❌ PROBLEM: Manual delay to avoid hitting rate limits
            time.sleep(1)
        
        # Final answer
        all_research = "\n\n=== RESEARCH ROUND ===\n\n".join(all_results)
        final_answer = self.answer_chain.run(question=question, all_results=all_research)
        
        return final_answer, iteration
    
    def attempt_user_interaction(self):
        """Try to create interactive research - shows LangChain's limitations"""
        print("🤖 LangChain Interactive Research (Attempt)")
        print("❌ Note: This is hacky and not how LangChain is designed to work!\n")
        
        # Get user question
        user_question = input("👤 What would you like to research? ")
        
        if not user_question.strip():
            print("Please provide a question!")
            return
        
        # Do initial research
        answer, iterations = self.research_with_manual_loops(user_question)
        
        print(f"\n🎯 Final Answer ({iterations} iterations):")
        print(answer)
        
        # ❌ PROBLEM: Can't easily loop back for follow-up questions
        # Would need to restart the entire process!
        print("\n❌ LangChain Limitation: Follow-up questions require starting over!")

# Demo LangChain limitations
lc_research = LangChainIterativeResearch()

print("=" * 60)
print("🔗 LANGCHAIN: Attempting Iterative Research")
print("=" * 60)

# Uncomment to try interactive mode:
# lc_research.attempt_user_interaction()

# For demo purposes, let's show with a hardcoded example:
demo_question = "Who is the president of usa"
demo_answer, demo_iterations = lc_research.research_with_manual_loops(demo_question, max_iterations=10)
print(f"\n🎯 Demo Result: {demo_answer[:300]}...")
print(f"📊 Required {demo_iterations} manual iterations")

### 😫 Problems with LangChain for Loops:

1. **No Native Loop Support** - Chains are designed to be linear
2. **Manual While Loops** - External control logic required (hacky!)
3. **Brittle Condition Checking** - Text parsing to determine if loop should continue
4. **No State Management** - Can't track iteration state naturally
5. **Poor Error Handling** - If one iteration fails, hard to recover
6. **No Conditional Routing** - Can't dynamically choose next steps
7. **Restart Required** - Follow-up questions need complete restart
8. **Complex Control Flow** - Business logic mixed with framework limitations

**🤔 The core issue**: LangChain chains are **directed acyclic graphs** (DAGs) - no cycles allowed!

# 🕸️ B. LangGraph Approach: Loops are Natural

## ✅ The Solution: Built-in Loop Support with State Management

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage

# Define research state that persists across loop iterations
class ResearchState(TypedDict):
    original_question: str
    current_query: str
    search_results: Annotated[list, operator.add]
    analysis_history: Annotated[list, operator.add]
    iteration_count: int
    research_complete: bool
    information_gaps: List[str]
    final_answer: str
    user_satisfied: bool
    follow_up_requested: str

# Node functions for the research workflow
def search_web(state: ResearchState):
    """Search the web for current query"""
    query = state["current_query"]
    iteration = state["iteration_count"]
    
    print(f"🔍 Iteration {iteration}: Searching for '{query}'")
    
    try:
        search_results = search_tool.invoke(query)
        formatted_results = {
            "query": query,
            "results": search_results,
            "formatted": "\n\n".join([
                f"Source: {result['url']}\n{result['content']}"
                for result in search_results
            ])
        }
    except Exception as e:
        print(f"⚠️ Search failed: {e}")
        formatted_results = {
            "query": query,
            "results": [],
            "formatted": f"Search failed: {e}"
        }
    
    return {
        "search_results": [formatted_results]
    }

def analyze_completeness(state: ResearchState):
    """Analyze if research is complete or needs more iterations"""
    question = state["original_question"]
    all_results = state["search_results"]
    iteration = state["iteration_count"]
    
    # Combine all research so far
    combined_research = "\n\n--- RESEARCH ROUND ---\n\n".join([
        result["formatted"] for result in all_results
    ])
    
    analysis_prompt = f"""Original Question: {question}

Research Completed So Far:
{combined_research}

Analysis Task:
1. Is this information sufficient to provide a comprehensive answer?
2. What specific information gaps remain?
3. If more research needed, what should be the next search query?

Respond in this format:
COMPLETE: true/false
GAPS: [list any remaining information gaps]
NEXT_QUERY: [specific search query for next iteration]
REASONING: [why more research is or isn't needed]"""
    
    analysis = llm.invoke([HumanMessage(content=analysis_prompt)]).content
    
    # Parse analysis (more robust than LangChain version)
    complete = "COMPLETE: true" in analysis.lower() or iteration >= 3  # Max 3 iterations
    
    # Extract information gaps
    gaps = []
    lines = analysis.split('\n')
    for line in lines:
        if line.startswith("GAPS:"):
            gaps_text = line.replace("GAPS:", "").strip()
            if gaps_text and gaps_text != "None":
                gaps = [gap.strip() for gap in gaps_text.split(",")]
    
    # Extract next query
    next_query = state["current_query"]  # Default fallback
    for line in lines:
        if line.startswith("NEXT_QUERY:"):
            next_query = line.replace("NEXT_QUERY:", "").strip()
            break
    
    print(f"📊 Analysis complete: Research {'finished' if complete else 'continuing'}")
    if not complete:
        print(f"🎯 Next query: {next_query}")
    
    return {
        "analysis_history": [{
            "iteration": iteration,
            "analysis": analysis,
            "complete": complete
        }],
        "research_complete": complete,
        "information_gaps": gaps,
        "current_query": next_query,
        "iteration_count": iteration + 1
    }

def generate_final_answer(state: ResearchState):
    """Generate comprehensive final answer from all research"""
    question = state["original_question"]
    all_results = state["search_results"]
    
    # Combine all research
    combined_research = "\n\n--- RESEARCH ROUND ---\n\n".join([
        f"Query: {result['query']}\n{result['formatted']}"
        for result in all_results
    ])
    
    final_prompt = f"""Question: {question}

Complete Research Results:
{combined_research}

Please provide a comprehensive, well-structured answer that synthesizes all the research findings. 
Include key points, examples, and cite sources where relevant."""
    
    final_answer = llm.invoke([HumanMessage(content=final_prompt)]).content
    
    print("✅ Final answer generated")
    
    return {
        "final_answer": final_answer
    }

def ask_user_satisfaction(state: ResearchState):
    """Check if user wants more details on any aspect"""
    answer = state["final_answer"]
    
    print("\n" + "=" * 60)
    print("🎯 RESEARCH COMPLETE")
    print("=" * 60)
    print(answer)
    print("\n" + "=" * 60)
    
    follow_up = input("\n👤 Would you like more detail on any specific aspect? (or 'done' to finish): ").strip()
    
    if follow_up.lower() in ['done', 'no', 'n', '']:
        satisfied = True
        follow_up = ""
    else:
        satisfied = False
    
    return {
        "user_satisfied": satisfied,
        "follow_up_requested": follow_up,
        "current_query": follow_up,  # New search query
        "research_complete": False if not satisfied else True,  # Reset if more research needed
        "iteration_count": 1 if not satisfied else state["iteration_count"]  # Reset counter for follow-up
    }

# Conditional routing functions
def should_continue_research(state: ResearchState):
    """Decide whether to continue research loop"""
    if state["research_complete"]:
        return "generate_answer"
    else:
        return "search"

def should_ask_user(state: ResearchState):
    """Decide whether to ask user for satisfaction"""
    return "ask_user" if state["research_complete"] else "continue"

def should_finish(state: ResearchState):
    """Decide whether to finish or continue with follow-up"""
    if state["user_satisfied"]:
        return "finish"
    else:
        return "search"  # Start new research cycle

def visualize_data():
    pass


# Build the research workflow graph
research_workflow = StateGraph(ResearchState)

# Add nodes
research_workflow.add_node("search", search_web)
research_workflow.add_node("analyze", analyze_completeness)
research_workflow.add_node("generate_answer", generate_final_answer)
research_workflow.add_node("ask_user", ask_user_satisfaction)


#Plannning     
research_workflow.add_edge(START, "search")
research_workflow.add_edge("search", "analyze")

# ✅ LOOP: analyze can route back to search!
research_workflow.add_conditional_edges(
    "analyze",
    should_continue_research,
    {
        "search": "search",  # ✅ LOOP BACK!
        "generate_answer": "generate_answer"
    }
)


research_workflow.add_edge("generate_answer", "ask_user")

# ✅ LOOP: user can request more research!
research_workflow.add_conditional_edges(
    "ask_user",
    should_finish,
    {
        "search": "search",  # ✅ LOOP BACK for follow-up!
        "finish": END
    }
)

# Compile the graph
research_app = research_workflow.compile()

print("🕸️ LangGraph iterative research workflow compiled with native loop support!")

In [ ]:
def interactive_research():
    """Interactive research session with natural loops"""
    print("🤖 Welcome to the LangGraph Iterative Research Assistant!")
    print("I'll conduct thorough research and ask follow-up questions as needed.")
    print("The system will automatically determine when more research is required.\n")
    
    # Get user's research question
    user_question = input("👤 What would you like me to research in depth? ")
    
    if not user_question.strip():
        print("Please provide a research question!")
        return
    
    # Initialize research state
    initial_state = {
        "original_question": user_question,
        "current_query": user_question,
        "search_results": [],
        "analysis_history": [],
        "iteration_count": 1,
        "research_complete": False,
        "information_gaps": [],
        "final_answer": "",
        "user_satisfied": False,
        "follow_up_requested": ""
    }
    
    print(f"\n🔬 Starting iterative research on: '{user_question}'")
    print("🔄 The system will automatically loop until comprehensive answer is found...\n")
    
    try:
        # Run the research workflow - loops automatically!
        final_state = research_app.invoke(initial_state)
        
        print("\n🎉 Research session completed!")
        print(f"📊 Total iterations: {final_state['iteration_count'] - 1}")
        print(f"🔍 Search queries used: {len(final_state['search_results'])}")
        
    except KeyboardInterrupt:
        print("\n👋 Research session interrupted by user.")
    except Exception as e:
        print(f"\n❌ Error during research: {e}")

# Uncomment to start interactive research:
interactive_research()

print("💡 Tip: Uncomment the line above to try interactive iterative research!")
print("\nSample research topics to try:")
print("1. 'What are the latest breakthroughs in quantum computing?'")
print("2. 'How is artificial intelligence being used in healthcare?'")
print("3. 'What are the environmental effects of electric vehicles?'")
print("4. 'How do social media algorithms affect mental health?'")

# Demo with a fixed example
print("\n" + "=" * 60)
print("🔬 DEMO: Automatic Iterative Research")
print("=" * 60)

demo_state = {
    "original_question": "What are the environmental impacts of AI and machine learning?",
    "current_query": "environmental impacts AI machine learning carbon footprint",
    "search_results": [],
    "analysis_history": [],
    "iteration_count": 1,
    "research_complete": False,
    "information_gaps": [],
    "final_answer": "",
    "user_satisfied": True,  # Skip user interaction for demo
    "follow_up_requested": ""
}

# Run a shorter demo version
print("Running automated research demo...")
final_demo = research_app.invoke(demo_state)

🔍 Iteration 1: Searching for 'how much electricity will another ai data center from big tech company use?'
📊 Analysis complete: Research continuing
🎯 Next query: "sustainable practices in AI data centers and their impact on carbon emissions"
🔍 Iteration 2: Searching for '"sustainable practices in AI data centers and their impact on carbon emissions"'
📊 Analysis complete: Research continuing
🎯 Next query: "case studies on sustainable AI practices in data centers"
🔍 Iteration 3: Searching for '"case studies on sustainable AI practices in data centers"'
📊 Analysis complete: Research finished
✅ Final answer generated

🎯 RESEARCH COMPLETE
### Environmental Impacts of AI and Machine Learning

The rise of artificial intelligence (AI) and machine learning (ML) technologies has brought significant advancements across various sectors, but it also poses notable environmental challenges. This synthesis examines the environmental impacts of AI, particularly focusing on its carbon footprint, energy 

KeyboardInterrupt: Interrupted by user

### ✅ Benefits of LangGraph for Loops:

1. **Native Loop Support** - Cycles are first-class citizens in the graph
2. **Conditional Routing** - Smart decisions about when to loop
3. **State Persistence** - Information accumulates across iterations
4. **Flexible Control Flow** - Can loop back from any node
5. **Error Recovery** - Failed iterations don't break the entire flow
6. **User Interaction** - Natural follow-up questions and refinement
7. **Termination Conditions** - Multiple ways to exit loops gracefully
8. **Visual Workflow** - Easy to understand and debug loop logic

**🧠 The key insight**: LangGraph treats **loops as natural workflow patterns**, not exceptional cases!

# 📊 Side-by-Side Comparison

## 🔗 LangChain Approach:
```python
# ❌ Manual while loop (external to framework)
while iteration < max_iterations:
    search_results = search_tool.invoke(query)
    analysis = analysis_chain.run(results=search_results)
    
    # ❌ Brittle text parsing
    if "SUFFICIENT: Yes" in analysis:
        break  # Manual loop control
    
    # ❌ Extract next query manually
    query = parse_next_query(analysis)

# Problems:
# - External loop control
# - Brittle condition checking
# - No state management
# - Hard to handle errors
```

## 🕸️ LangGraph Approach:
```python
# ✅ Natural loop as part of workflow
workflow.add_conditional_edges(
    "analyze",
    should_continue_research,
    {
        "search": "search",      # ✅ Loop back!
        "generate_answer": "generate_answer"
    }
)

workflow.add_conditional_edges(
    "ask_user",
    should_finish,
    {
        "search": "search",      # ✅ Follow-up loop!
        "finish": END
    }
)

# Benefits:
# - Native loop support
# - Declarative conditions
# - Automatic state management
# - Built-in error handling
```

# 🎯 When Do You Need Loops?

## 🔄 **Common Loop Patterns in AI:**

### 1. **Iterative Refinement**
- Research that builds on previous findings
- Code generation with debugging cycles
- Creative writing with revision loops

### 2. **Progressive Elaboration**
- Start with high-level overview
- Drill down into specific details
- User-guided exploration

### 3. **Trial and Error**
- Multiple search strategies
- Fallback approaches when primary fails
- A/B testing different prompts

### 4. **User Interaction Cycles**
- Chatbot conversations
- Interactive tutorials
- Multi-turn decision making

### 5. **Conditional Processing**
- "Keep searching until satisfied"
- "Try different approaches until success"
- "Refine until quality threshold met"

## 🚫 **When You DON'T Need Loops:**
- Simple question → answer workflows
- One-time document processing
- Static data transformations
- Batch processing independent items

# 🎓 Key Takeaways

## 🔍 **The Fundamental Difference**:
- **LangChain**: Linear chains (DAGs) - **no cycles allowed**
- **LangGraph**: State machines - **loops are natural**

## 🔄 **Loop Implementation**:
- **LangChain**: External while loops (hacky, brittle)
- **LangGraph**: Conditional edges (elegant, robust)

## 🧠 **Mental Models**:
- **LangChain**: Assembly line - each step happens once
- **LangGraph**: Workflow engine - steps can repeat as needed

## 🎯 **The Decision Questions**:
1. **Do you need to repeat steps based on conditions?** → LangGraph
2. **Will users want to refine or iterate?** → LangGraph
3. **Is it a simple linear process?** → LangChain
4. **Do you need "keep trying until success"?** → LangGraph

## 💡 **Pro Tips**:
- **LangChain** is perfect for **linear, one-shot workflows**
- **LangGraph** shines when you need **iterative, adaptive processes**
- **Loops** are essential for **research, refinement, and interaction**
- **State management** makes complex workflows much easier

## 🚀 **Real-World Examples**:
- **Research Assistant**: Keep searching until comprehensive
- **Code Debugger**: Try fixes until tests pass
- **Creative Writer**: Refine until user satisfied
- **Data Analyst**: Explore until insights found
- **Tutor**: Explain until student understands

## 🔮 **The Future is Iterative**:
AI workflows are becoming more **sophisticated** and **interactive**. The ability to **loop, adapt, and refine** is crucial for building truly intelligent systems.

**Remember**: If your AI needs to **"keep trying until..."** - you need LangGraph! 🔄